# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [11]:
#Loading enviroment viariables
%load_ext dotenv
%dotenv 

#ignore warnings
import warnings
warnings.filterwarnings('ignore')


The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [12]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [13]:
import os
from glob import glob
import pandas as pd

#Loading the 'PRICE_DATA' variable  
PRICE_DATA = os.getenv("PRICE_DATA")

#Finding the path of all parquet files in the directory PRiCE_DATA
parquet_files = glob(os.path.join(PRICE_DATA, "*/*/*.parquet"))


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [14]:
#Adding parquet files to dd dataframe
dd_px = dd.read_parquet(parquet_files)
dd_px.head()

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year
ticker,,,,,,,,,,
A,2000-01-03,56.330471,56.464592,48.193848,51.502148,43.463043,4674353,Health Care,Life Sciences Tools & Services,2000
A,2000-01-04,48.730328,49.266811,46.316166,47.567955,40.142937,4765083,Health Care,Life Sciences Tools & Services,2000
A,2000-01-05,47.389126,47.567955,43.141991,44.617310,37.652859,5758642,Health Care,Life Sciences Tools & Services,2000
A,2000-01-06,44.080830,44.349072,41.577251,42.918453,36.219185,2534434,Health Care,Life Sciences Tools & Services,2000
A,2000-01-07,42.247852,47.165592,42.203148,46.494991,39.237469,2819626,Health Care,Life Sciences Tools & Services,2000


In [15]:
dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1))
).assign(
    returns = lambda x: x['Close']/x['Close_lag_1'] - 1
)).assign(
    hi_lo_range = lambda x: x['High'] - x['Low']
)


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [16]:
df_feat = dd_feat.compute()

In [17]:
df_feat['rolling_average'] = df_feat.groupby('ticker')['returns'].transform(lambda x: x.rolling(10).mean())


In [18]:
df_feat.head(20)

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Close_lag_1,returns,hi_lo_range,rolling_average
ticker,,,,,,,,,,,,,,
HUM,2006-01-03,54.910000,55.779999,54.099998,55.779999,49.805698,2689400,Health Care,Managed Health Care,2006,NaN,NaN,1.680000,NaN
HUM,2006-01-04,56.200001,57.639999,56.180000,56.869999,50.778965,2334200,Health Care,Managed Health Care,2006,55.779999,0.019541,1.459999,NaN
HUM,2006-01-05,57.000000,57.880001,56.930000,57.459999,51.305763,1911200,Health Care,Managed Health Care,2006,56.869999,0.010375,0.950001,NaN
HUM,2006-01-06,57.400002,57.500000,56.570000,57.080002,50.966484,1991700,Health Care,Managed Health Care,2006,57.459999,-0.006613,0.930000,NaN
HUM,2006-01-09,57.090000,57.450001,56.360001,57.330002,51.189709,1874500,Health Care,Managed Health Care,2006,57.080002,0.004380,1.090000,NaN
HUM,2006-01-10,57.340000,57.750000,56.500000,56.709999,50.636120,1433000,Health Care,Managed Health Care,2006,57.330002,-0.010815,1.250000,NaN
HUM,2006-01-11,56.799999,58.070000,56.759998,56.770000,50.689686,1488800,Health Care,Managed Health Care,2006,56.709999,0.001058,1.310001,NaN
HUM,2006-01-12,56.430000,57.240002,56.009998,56.849998,50.761116,924400,Health Care,Managed Health Care,2006,56.770000,0.001409,1.230003,NaN
HUM,2006-01-13,56.480000,56.840000,55.709999,56.529999,50.475372,1754700,Health Care,Managed Health Care,2006,56.849998,-0.005629,1.130001,NaN


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

In [19]:
#It was not necessary to convert to pandas to calculate the moving average return as that operation can be completed
#similarly to how 'returns' and 'Hi_Low_Range' were calculated. See demonstration below. It is faster to run in dask as
#converting to pandas took approximately 2 and a half minutes. 

#Calculating the rolling average without converting
dd_feat['rolling_average'] = dd_feat.groupby('ticker')['returns'].transform(lambda x: x.rolling(10).mean())

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.